In [6]:
import ufl
import numpy

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

def q(u):
    return 1 + u**2


domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
x = ufl.SpatialCoordinate(domain)
u_ufl = 1 + x[0] + 2 * x[1]
f = - ufl.div(q(u_ufl) * ufl.grad(u_ufl))

In [7]:
V = fem.functionspace(domain, ("Lagrange", 1))
def u_exact(x): return eval(str(u_ufl))

In [8]:
u_D = fem.Function(V)
u_D.interpolate(u_exact)
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets))

In [9]:
uh = fem.Function(V)
v = ufl.TestFunction(V)
F = q(uh) * ufl.dot(ufl.grad(uh), ufl.grad(v)) * ufl.dx - f * v * ufl.dx

In [10]:
problem = NonlinearProblem(F, uh, bcs=[bc])

In [11]:
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True

In [12]:
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}ksp_rtol"] = 1.0e-8
opts[f"{option_prefix}pc_type"] = "hypre"
opts[f"{option_prefix}pc_hypre_type"] = "boomeramg"
opts[f"{option_prefix}pc_hypre_boomeramg_max_iter"] = 1
opts[f"{option_prefix}pc_hypre_boomeramg_cycle_type"] = "v"
ksp.setFromOptions()

In [13]:
log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(uh)
assert (converged)
print(f"Number of interations: {n:d}")

[2025-04-11 10:42:20.478] [info] PETSc Krylov solver starting to solve system.
Number of interations: 8
[2025-04-11 10:42:20.483] [info] PETSc Krylov solver starting to solve system.
[2025-04-11 10:42:20.484] [info] Newton iteration 2: r (abs) = 20.37916572634953 (tol = 1e-10), r (rel) = 0.9225323398510273 (tol = 1e-06)
[2025-04-11 10:42:20.484] [info] PETSc Krylov solver starting to solve system.
[2025-04-11 10:42:20.484] [info] Newton iteration 3: r (abs) = 6.952713011092108 (tol = 1e-10), r (rel) = 0.31473823259321554 (tol = 1e-06)
[2025-04-11 10:42:20.485] [info] PETSc Krylov solver starting to solve system.
[2025-04-11 10:42:20.485] [info] Newton iteration 4: r (abs) = 2.935703719836482 (tol = 1e-10), r (rel) = 0.13289462670537117 (tol = 1e-06)
[2025-04-11 10:42:20.485] [info] PETSc Krylov solver starting to solve system.
[2025-04-11 10:42:20.485] [info] Newton iteration 5: r (abs) = 0.700589737775592 (tol = 1e-10), r (rel) = 0.03171458040748303 (tol = 1e-06)
[2025-04-11 10:42:20.

In [14]:
# Compute L2 error and error at nodes
V_ex = fem.functionspace(domain, ("Lagrange", 2))
u_ex = fem.Function(V_ex)
u_ex.interpolate(u_exact)
error_local = fem.assemble_scalar(fem.form((uh - u_ex)**2 * ufl.dx))
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))
if domain.comm.rank == 0:
    print(f"L2-error: {error_L2:.2e}")

# Compute values at mesh vertices
error_max = domain.comm.allreduce(numpy.max(numpy.abs(uh.x.array - u_D.x.array)), op=MPI.MAX)
if domain.comm.rank == 0:
    print(f"Error_max: {error_max:.2e}")

[2025-04-11 10:42:35.478] [info] Checking required entities per dimension
[2025-04-11 10:42:35.478] [info] Cell type: 0 dofmap: 200x6
[2025-04-11 10:42:35.478] [info] Global index computation
[2025-04-11 10:42:35.478] [info] Got 2 index_maps
[2025-04-11 10:42:35.478] [info] Get global indices
L2-error: 6.30e-16
Error_max: 4.44e-16


In [19]:
solver.help

AttributeError: 'NewtonSolver' object has no attribute 'help'